In [8]:
import os
import json
import re
import shutil

In [9]:
# Dicionário de Regex para Marcas
# Adicionei suas regras novas aqui já
car_to_manufacturer = {
    r"\bBrabham\b": "Brabham",
    r"\bCorvette\b": "Chevrolet",
    r"\bChevelle\b": "Chevrolet",
    r"\bChevy\b": "Chevrolet",
    r"\bCamaro\b": "Chevrolet",
    r"\bImpala\b": "Chevrolet",
    r"\bMustang\b": "Ford", 
    r"\bEscort\b": "Ford",
    r"\bChallenger\b": "Dodge",
    r"\bCharger\b": "Dodge",
    r"\bViper\b": "Dodge",
    r"\bRam\b": "Dodge",
    r"\bLancer\b": "Mitsubishi",
    r"\bVW\b": "Volkswagen",      
    r"\bVolkswagen\b": "Volkswagen",
    r"\bPorsche\b": "Porsche",
    r"\bMercedes\b": "Mercedes-Benz",
    r"\bBenz\b": "Mercedes-Benz"
}

# Lista de Fabricantes Reais para validação
real_manufacturers = [
    "Acura", "Alfa Romeo", "Aston Martin", "Audi", "Bentley", "BMW", "Brabham", "Bugatti", "Buick", "Cadillac", "Citroen", "Chevrolet", "Chevy", "Datsun", "Dodge", "Ducati", "Ferrari", "Fiat", "Ford", "GMC", "Honda", "Jaguar", "Jeep", "Kia", "Koenigsegg", "Lamborghini", "Lancia", "Land Rover", "Lincoln", "Lotus", "Maserati", "Mercedes", "Mercury", "Mazda", "McLaren", "Mini", "Mitsubishi", "Nissan", "Pagani", "Peugeot", "Plymouth", "Pontiac", "Porsche", "Range Rover", "Renault", "Rimac", "Shelby", "Subaru", "Tesla", "Toyota", "Volvo", "Volkswagen"
]

print("✅ Regras de correção carregadas!")

✅ Regras de correção carregadas!


In [10]:
def slugify(text):
    """Converte texto para slug URL-friendly"""
    if not text: return "unknown"
    return re.sub(r'[^a-z0-9]+', '-', text.lower()).strip('-')

def extract_manufacturer(casting_name):
    """Extrai fabricante do nome do casting"""
    if not casting_name: return "Unknown"

    # 1. Tenta pelo Regex (Prioridade Máxima)
    for pattern, manufacturer in car_to_manufacturer.items():
        if re.search(pattern, casting_name, re.IGNORECASE):
            return manufacturer
    
    # 2. Tenta pela Primeira Palavra exata
    first_word = casting_name.split()[0]
    for manufacturer in real_manufacturers:
        if manufacturer.lower() == first_word.lower():
            return manufacturer
    
    # 3. Tenta encontrar a marca no meio do nome
    for m in real_manufacturers:
        if m.lower() in casting_name.lower(): 
            return m
    
    return "Mattel"
    
print("✅ Funções compiladas!")

✅ Funções compiladas!


In [ ]:
def fix_brand_castings():
    brands_dir = 'data/brands'
    castings_moved = []
    
    print("🔍 Iniciando auditoria das marcas...")
    
    # Carregar arquivos
    brand_files = {}
    if os.path.exists(brands_dir):
        for filename in os.listdir(brands_dir):
            if filename.endswith('.json'):
                filepath = os.path.join(brands_dir, filename)
                with open(filepath, 'r', encoding='utf-8') as f:
                    brand_files[filename[:-5]] = json.load(f)
    else:
        print("❌ Pasta data/brands não encontrada. Rode o organize.ipynb primeiro.")
        return

    print(f"📂 Analisando {len(brand_files)} marcas carregadas...")
    
    # Cópia para iterar com segurança
    brand_files_copy = brand_files.copy()
    
    for brand_name, brand_data in brand_files_copy.items():
        castings_to_remove = []
        
        for casting_path in brand_data.get('castings', []):
            # Analisa o nome
            casting_filename = casting_path.split('/')[-1]
            casting_name = casting_filename[:-5]
            
            # Limpa o nome para leitura (tira números do ID)
            readable_name = re.sub(r'^\d+-', '', casting_name).replace('-', ' ')
            readable_name = ' '.join(word.capitalize() for word in readable_name.split())
            
            # Descobre quem é o dono de verdade
            correct_manufacturer = extract_manufacturer(readable_name)
            correct_brand_slug = slugify(correct_manufacturer)
            
            # SE ESTIVER NO LUGAR ERRADO
            if correct_brand_slug != brand_name:
                
                # Prepara a nova marca se não existir
                if correct_brand_slug not in brand_files:
                    brand_files[correct_brand_slug] = {
                        "brand_id": correct_brand_slug,
                        "name": correct_manufacturer,
                        "country": "Unknown",
                        "founded_year": None,
                        "castings": [],
                        "description": {
                            "en-us": f"Manufacturer of {correct_manufacturer} vehicles.",
                            "pt-br": f"Fabricante de veículos {correct_manufacturer}."
                        }
                    }
                    print(f"✨ NOVA MARCA DETECTADA: {correct_manufacturer} ({correct_brand_slug}.json)")
                
                # Move o registro
                if casting_path not in brand_files[correct_brand_slug]['castings']:
                    brand_files[correct_brand_slug]['castings'].append(casting_path)
                    castings_moved.append(f"{readable_name}: Saiu de '{brand_name}' -> Foi para '{correct_brand_slug}'")
                
                # Marca para tirar da antiga
                castings_to_remove.append(casting_path)
        
        # Remove da lista antiga
        for casting_path in castings_to_remove:
            brand_data['castings'].remove(casting_path)
            
        # Se a marca ficou vazia (ex: Beatnik), deletamos o arquivo dela?
        # Por segurança, vamos manter o arquivo mas sem castings por enquanto, 
        # ou você pode adicionar uma lógica aqui para apagar brand_files[brand_name] se castings estiver vazio.

    # Salvar tudo
    if castings_moved:
        print(f"\n💾 Salvando alterações em {len(brand_files)} arquivos...")
        for brand_name, brand_data in brand_files.items():
            # Ordena e salva
            brand_data['castings'].sort()
            filepath = os.path.join(brands_dir, f'{brand_name}.json')
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(brand_data, f, indent=4, ensure_ascii=False)
        
        print(f"\n✅ SUCESSO! {len(castings_moved)} carros foram movidos para as marcas certas.")
        for move in castings_moved:
            print(f"   🔄 {move}")
    else:
        print("\n🎉 Tudo limpo! Nenhum carro estava na marca errada.")

# Executa
fix_brand_castings()

🔍 Iniciando auditoria das marcas...
📂 Analisando 12 marcas carregadas...

🎉 Tudo limpo! Nenhum carro estava na marca errada.
